In [1]:
import warnings
warnings.filterwarnings("ignore")
from numpy import cos, sin, arcsin, sqrt
from math import radians
import math
import pandas as pd
import numpy as np
import geocoder
from tqdm import tqdm

In [2]:
import folium
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [3]:
vsp = pd.read_csv('VSP_LIST.csv')

In [4]:
res = [geocoder.arcgis(x).latlng for x in tqdm(vsp['адрес'].tolist()) ]

100%|████████████████████████████████████████████████████████████████████████████████| 713/713 [09:40<00:00,  1.23it/s]


In [5]:
vsp['широта'], vsp['долгота'] = zip(*res)

In [6]:
def search_distance(lat1, lon1, lat2, lon2):
       
    rlat1 = lat1*math.pi/180.
    rlat2 = lat2*math.pi/180.
    rlon1 = lon1*math.pi/180.
    rlon2 = lon2*math.pi/180.
    
    cosl1 = math.cos(rlat1)
    cosl2 = math.cos(rlat2)
    sinl1 = math.sin(rlat1)
    sinl2 = math.sin(rlat2)
    
    dlon = rlon2 - rlon1 
    dlat = rlat2 - rlat1
    
    a = sin(dlat/2)**2 + cosl1 * cosl2 * sin(dlon/2)**2
    r = 2*6372795*math.asin(sqrt(a))
    
    return r

In [7]:
res = []
lsts=vsp[['адрес', 'широта', 'долгота']].values
for i, obj1 in enumerate(lsts):
    for adress2, lat2, lon2 in lsts[i+1:-1]:
        distance= search_distance(obj1[1], obj1[2], lat2, lon2)
        res.append([obj1[0], adress2, obj1[1], obj1[2], lat2, lon2, distance])

In [8]:
vsp2 = pd.DataFrame(data=res, columns=['адрес1','адрес2','широта1', 'долгота1', 'широта2', 'долгота2', 'расстояние'])

In [9]:
distance_vsp = vsp2[(vsp2['расстояние']<250) & (vsp2['расстояние']!=0)]

In [10]:
distance_vsp.sample(5)

,адрес1,адрес2,широта1,долгота1,широта2,долгота2,расстояние
170117,"г Москва, б-р Новочеркасский, д 46","г Москва, б-р Новочеркасский, д 41 к 2",55.64611,37.73831,55.648130,37.737250,234.320883
143090,"г Москва, ул Хачатуряна, д 16","г Москва, ул Хачатуряна, д 20",55.86053,37.60297,55.861970,37.604440,184.589055
242676,"г Санкт-Петербург, пр-кт Косыгина, д 27 к 1","г Санкт-Петербург, пр-кт Косыгина, д 28 к 1",59.94471,30.49519,59.945739,30.496531,136.662821
231826,"г Санкт-Петербург, г Колпино, пр-кт Ленина, д 37","г Санкт-Петербург, г Красное Село, пр-кт Ленин...",59.82668,30.08178,59.826044,30.081599,71.480248
220615,"г Санкт-Петербург, ул 2-я Красноармейская, д 2/27","г Санкт-Петербург, ул 1-я Красноармейская, д 4...",59.91518,30.31737,59.915890,30.316400,95.714710


In [17]:
russia_map = folium.Map(
    location = [55.755864, 37.617698],
    zoom_start = 6)

In [12]:
dist_vsp1 = pd.DataFrame()
dist_vsp2 = pd.DataFrame()
dist_vsp1[['адрес', 'широта', 'долгота']]=distance_vsp[['адрес1', 'широта1', 'долгота1']].values
dist_vsp2[['адрес', 'широта', 'долгота']]=distance_vsp[['адрес2', 'широта2', 'долгота2']].values

In [13]:
pp = pd.concat([dist_vsp1, dist_vsp2]).drop_duplicates(subset=['адрес'])

In [19]:
#lat = distance_vsp['широта1']
#lon = distance_vsp['долгота1']
#adress = distance_vsp['адрес1']

#for adress, lat, lon in pp.values: #zip(lat, lon, adress):
    #print(f'{lat}-jghbhgu{lon}-ghbjkk{adress}')
    #folium.Marker(location=[lat, lon], popup=str(adress), icon=folium.Icon(color = 'gray')).add_to(russia_map)

In [20]:
marker_cluster = MarkerCluster().add_to(russia_map)

for adress, lat, lon in pp.values:
    folium.CircleMarker(location=[lat, lon], radius = 9, popup=str(adress), fill_color='blue', icon=folium.Icon(color = 'blue'), fill_opacity = 0.9).add_to(marker_cluster)

In [21]:
russia_map.save('russia_map.html')

In [22]:
russia_map